# Protocol Workflow Extraction Tool   

> Langchain tools that execute zero-shot extraction over a local database of full text papers previously imported into our database.

In [7]:
#| default_exp tools.protocol_extraction_tool

In [2]:
#| hide
from nbdev import *

In [1]:
#| export

from alhazen.core import OllamaRunner, PromptTemplateRegistry, get_langchain_llm, get_cached_gguf, \
    get_langchain_embeddings, GGUF_LOOKUP_URL, MODEL_TYPE, load_alhazen_tool_environment, get_langchain_chatmodel
from alhazen.tools.basic import AlhazenToolMixin
from alhazen.utils.output_parsers import *
from alhazen.utils.ceifns_db import *
from alhazen.schema_sqla import *
from datetime import datetime
from importlib_resources import files
import jmespath
import json

from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain.schema import get_buffer_string, StrOutputParser, OutputParserException, format_document
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.tools import BaseTool, StructuredTool
from langchain.vectorstores.pgvector import PGVector

import local_resources.prompt_elements as prompt_elements
import local_resources.linkml as linkml
from operator import itemgetter
import os
import pandas as pd
import re
import regex
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker, aliased
from time import time,sleep
from urllib.request import urlopen
from urllib.parse import quote_plus, quote, unquote
import uuid
import yaml

In [2]:
#| export 

skc = aliased(ScientificKnowledgeCollection)
skc_hm = aliased(ScientificKnowledgeCollectionHasMembers)
ske = aliased(ScientificKnowledgeExpression)
ske_hr = aliased(ScientificKnowledgeExpressionHasRepresentation)
ski = aliased(ScientificKnowledgeItem)
ski_hp = aliased(ScientificKnowledgeItemHasPart)
skf = aliased(ScientificKnowledgeFragment)
n = aliased(Note)
skc_hn = aliased(ScientificKnowledgeCollectionHasNotes)
ske_hn = aliased(ScientificKnowledgeExpressionHasNotes)
ski_hn = aliased(ScientificKnowledgeItemHasNotes)
skf_hn = aliased(ScientificKnowledgeFragmentHasNotes)

In [3]:
#| export

class ProtocolExtractionToolSchema(BaseModel):
    paper_id: str = Field(description="the doi of the paper being analyzed, must start with the string 'doi:'")
    extraction_type: str = Field(description="This is the name of the type of extraction and must be one of the following strings: ['cryoet']")

class BaseProtocolExtractionTool(BaseTool, AlhazenToolMixin):
    '''Runs a specified protocol extraction pipeline over a research paper that has been loaded in the local literature database.'''
    name = 'protocol_workflow_extraction'
    description = 'Runs a specified protocol extraction pipeline over a research paper that has been loaded in the local literature database.'
    args_schema = ProtocolExtractionToolSchema
    return_direct:bool = True

    def _run(self, paper_id, extraction_type):
        '''Runs a specified protocol extraction pipeline over a research paper that has been loaded in the local literature database.'''
        raise NotImplementedError('This method must be implemented by a subclass.')

In [4]:
#| export
class ProcotolExtractionTool(BaseProtocolExtractionTool):
    '''Runs a specified metadata extraction pipeline over a research paper that has been loaded in the local literature database.'''
    name = 'protocol_extraction_over_methods_section'
    description = 'Runs a specified metadata extraction pipeline over a research paper that has been loaded in the local literature database.'

    def _run(self, paper_id, extraction_type):
        '''Runs the metadata extraction pipeline over a specified paper.'''

        if self.db.session is None:
            session_class = sessionmaker(bind=self.db.engine)
            self.db.session = session_class()

        ske = self.db.session.query(ScientificKnowledgeExpression) \
                .filter(ScientificKnowledgeExpression.id.like('%'+paper_id+'%')).first()    

        # Introspect the class name of the llm model for notes and logging
        llm_class_name = self.llm.__class__.__name__
        llm_model_desc = str(self.llm)

        run_name = self.__class__.__name__ + '__' + re.sub(' ','_',extraction_type) + '__' + paper_id + '__' + llm_class_name + '__' + llm_model_desc

        # 0. Use the first available full text item type
        item_types = set()
        item_type = None
        for i in self.db.list_items_for_expression(paper_id):
            item_types.add(i.type)
        for i_type in item_types:
            if i_type == 'CitationRecord':
                continue
            item_type = i_type
            break
        if item_type is None:
            return {'report': "Could not retrieve full text of the paper: %s."%(paper_id),
                "data": {'list_of_answers': None, 'run_name': run_name} }

        # 1. Build LangChain elements
        pts = PromptTemplateRegistry()
        
        pts.load_prompts_from_yaml('protocol_extraction.yaml')
        prompt_elements_yaml = files(prompt_elements).joinpath('metadata_extraction.yaml').read_text()
        prompt_elements_dict = yaml.safe_load(prompt_elements_yaml).get(extraction_type)
        all_protocol_steps = prompt_elements_dict['all protocol steps']
        all_entities = prompt_elements_dict['all entities']
        method_goal = prompt_elements_dict['method goal']
        methodology = prompt_elements_dict['methodology']
        metadata_specs = prompt_elements_dict.get('metadata specs',[])
        metadata_extraction_prompt_template = pts.get_prompt_template('protocol diagram extraction').generate_chat_prompt_template()
        extract_lcel = metadata_extraction_prompt_template | self.llm | MermaidExtractionOutputParser()
        
        # 2. Run through all available sections of the paper and identify only those that are predominantly methods sections.
        start = datetime.now()
        text = '\n\n'.join([f.content for f in self.db.list_fragments_for_paper(paper_id, item_type, fragment_types=['section'])])

        if len(text) == 0:
            return {'report': "Could not retrieve full text of the paper: %s."%(paper_id),
                "data": {'list_of_answers': None, 'run_name': run_name} }

        # 4. Assemble chain input
        s1 = {'section_text': text,
                'methodology': methodology,
                'method_goal': method_goal}
        
        # 5. Run the chain with a MermaidExtractionOutputParser 
        output = None
        full_answer = []
        output = extract_lcel.invoke(s1, config={'callbacks': [ConsoleCallbackHandler()]})
        total_execution_time = datetime.now() - start
        time_per_variable = total_execution_time / len(metadata_specs)

        if output is None:
            return {'report': "attempted and failed protocol extraction for an experiment of type '%s' from %s."%(methodology, paper_id),
                "data": {'mermaid_code': None, 'run_name': run_name} }
        
        return {'report': "completed protocol extraction for an experiment of type '%s' from %s."%(methodology, paper_id),
                "data": {'mermaid_code': output, 'run_name': run_name} }